In [1]:
import time,serial,random
import numpy as np

'''
Receive acc.txt
Receive prediction.txt
Backprop alg
Update weight.txt
Update bias.txt
Correct acc.txt to be appropriate
Backward pass with initials acc.txt to angles of matrix.txt
Update matrix.txt
'''
def sigmoid(a):
    return 1.0/(1.0+np.exp(a))

def rev_sigmoid(a):
    return np.log(1-a) - np.log(a+0.000000000000000000000000000001)

def line(mtrx):
    return mtrx.reshape(1,mtrx.shape[0] * mtrx.shape[1])

def backpropagation(mtrx,layr,prdcn,ac,w_2,w_1,b_2,b_1,alpha,lambd):
    mtrx = mtrx / 180.0
    ac = ac / 40.0 + 0.5
    prdcn = prdcn / 40.0 + 0.5

    J = np.sum((ac-prdcn)**2) + np.sum(w_1**2) + np.sum(w_2**2)
    with open('historia.log', 'a') as myfile:
            myfile.write('Target function ='+str(J)+'\nRunnning backpropagation.\n')
    delta_error = (prdcn - ac) * prdcn * (1 - prdcn)
    delta_hidden = (line(delta_error).dot(w_2.T)) * line(layr) * (1 - line(layr))
    delta_input = (line(delta_hidden).dot(w_1.T)) * line(mtrx) * (1 - line(mtrx))
      
    w_2 = w_2 - alpha * line(layer).T.dot(line(delta_error)) - lambd * w_2 
    w_1 = w_1 - alpha * line(matrix).T.dot(line(delta_hidden)) - lambd * w_1 
    b_2 = b_2 - alpha * delta_error
    b_1 = b_1 - alpha * delta_hidden.reshape(bias_1.shape) 
    return w_2,w_1,b_2,b_1

def zero_filter(mtrx,n):
    a = mtrx.shape[0] // n
    b = mtrx.shape[1] // n
    for i in range(n-1):
        for j in range(i+1,n):
            for k in range(a):
                for l in range(b):
                    mtrx[a*i+k][b*j+l] = 0  
    return mtrx

def correct(a):
    'gets real measurements & returns a which not fallen'
    return a

def backward_pass(mtrx,weight,bias):
    return (rev_sigmoid(line(mtrx))-line(bias)).dot(np.linalg.inv(weight))
        

if __name__ == "__main__":
        matrix = np.loadtxt('matrix.txt', 'int', delimiter='\t')
        acc = np.loadtxt('acc.txt', 'float', delimiter='\t')
        prediction = np.loadtxt('prediction.txt', 'float', delimiter='\t')
        layer = np.loadtxt('layer.txt', 'float', delimiter='\t')
        weight_1 = np.loadtxt('weight_1.txt', 'float', delimiter='|')
        weight_2 = np.loadtxt('weight_2.txt', 'float', delimiter='|')
        bias_1 = np.loadtxt('bias_1.txt', 'float', delimiter='\t')
        bias_2 = np.loadtxt('bias_2.txt', 'float', delimiter='\t')
        
        learning_rate = 0.1
        hyper = 0.2
        weight_2,weight_1,bias_2,bias_1 = backpropagation(matrix,layer,prediction,acc,weight_2,weight_1,bias_2,bias_1,learning_rate,hyper)
        weight_2 = zero_filter(weight_2,10)
        weight_1 = zero_filter(weight_1,10)
        
        np.savetxt('weight_2.txt',weight_2,fmt='%.2f',delimiter='|')
        np.savetxt('weight_1.txt',weight_1,fmt='%.2f',delimiter='|')
        np.savetxt('bias_2.txt',bias_2,fmt='%.2f',delimiter='\t')
        np.savetxt('bias_1.txt',bias_1,fmt='%.2f',delimiter='\t')
        
        acc = correct(acc)
        
        acc[:,0:3] = acc[:,0:3]  / 40.0 + 0.5
        acc[:,3:] = acc[:,3:]  / 500.0 + 0.5
        layer = backward_pass(acc,weight_2,bias_2) 
        matrix = backward_pass(layer,weight_1,bias_1).reshape(matrix.shape) * 180.0
        np.savetxt('matrix.txt',matrix,fmt='%d',delimiter='\t')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in log


ValueError: cannot convert float NaN to integer